## Действительно ли помогают неразмеченные данные?

Частичное обучение (semi-supervised learning) предлагает методы работы с выборками, в которых лишь для части объектов известны ответы. В статьях утверждается, что добавление неразмеченных данных позволяет повысить качество работы — давайте выясним, так ли это!

Наверное, проще всего добыть неразмеченные примеры, если речь идёт о работе с текстами или изображениями. Остановимся на текстах.

Будем работать с данными из соревнования Predict closed questions on Stack Overflow: https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data

Нас будет интересовать файл train-sample.csv — загрузите его. Будем решать бинарную задачу: отнесём объект к классу 1, если `OpenStatus == 'open'`, и к классу 0 иначе.

**Задание 1. (5 баллов)**

Загрузите данные и подготовьте выборку. В качестве признаков возьмите TF-IDF по BodyMarkdown с `min_df=10`; про целевую переменную написано выше. Выделите тестовую выборку из 5000 объектов.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
data = pd.read_csv("train-sample.csv")
assert('OpenStatus' in data.columns)
data['BodyMarkdown'] = data['BodyMarkdown'].fillna('')

y = (data['OpenStatus'] == 'open').astype(int)

vectorizer = TfidfVectorizer(min_df=10)
X_tfidf = vectorizer.fit_transform(data['BodyMarkdown'])

X_pool, X_test, y_pool, y_test = train_test_split(
    X_tfidf, y, test_size=5000, random_state=42, stratify=y
)

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import roc_auc_score
from scipy.sparse import vstack
base_lr = LogisticRegression(max_iter=1000, random_state=42)

print("1. Обучение только на размеченных данных (Supervised)")
model_1 = LogisticRegression(max_iter=1000, random_state=42)
model_1.fit(X_pool, y_pool)
probs_1 = model_1.predict_proba(X_test)[:, 1]
auc_1 = roc_auc_score(y_test, probs_1)
print(f"AUC-ROC: {auc_1:.4f}")

print("2. Semi-supervised (Unlabeled из Train, Test отделен)")

X_labeled, X_unlabeled, y_labeled, _ = train_test_split(
    X_pool, y_pool, test_size=0.5, random_state=42, stratify=y_pool
)

y_unlabeled_mock = np.full(X_unlabeled.shape[0], -1)

X_mix_2 = vstack([X_labeled, X_unlabeled])
y_mix_2 = np.concatenate([y_labeled, y_unlabeled_mock])

self_training_model_2 = SelfTrainingClassifier(base_lr, threshold=0.8, max_iter=100)
self_training_model_2.fit(X_mix_2, y_mix_2)
probs_2 = self_training_model_2.predict_proba(X_test)[:, 1]
auc_2 = roc_auc_score(y_test, probs_2)
print(f"AUC-ROC: {auc_2:.4f}")

print("3. Semi-supervised (Unlabeled == Test)")

X_labeled_3 = X_pool
y_labeled_3 = y_pool

X_unlabeled_3 = X_test
y_unlabeled_mock_3 = np.full(X_test.shape[0], -1)

X_mix_3 = vstack([X_labeled_3, X_unlabeled_3])
y_mix_3 = np.concatenate([y_labeled_3, y_unlabeled_mock_3])

self_training_model_3 = SelfTrainingClassifier(base_lr, threshold=0.8, max_iter=100)
self_training_model_3.fit(X_mix_3, y_mix_3)
probs_3 = self_training_model_3.predict_proba(X_test)[:, 1]
auc_3 = roc_auc_score(y_test, probs_3)
print(f"AUC-ROC: {auc_3:.4f}")

print("Сценарий 4: Semi-supervised (Unlabeled включает Test)")
X_labeled_4, X_rest_pool, y_labeled_4, _ = train_test_split(
    X_pool, y_pool, test_size=0.8, random_state=42, stratify=y_pool
)

X_unlabeled_4 = vstack([X_rest_pool, X_test])
y_unlabeled_mock_4 = np.full(X_unlabeled_4.shape[0], -1)

X_mix_4 = vstack([X_labeled_4, X_unlabeled_4])
y_mix_4 = np.concatenate([y_labeled_4, y_unlabeled_mock_4])

self_training_model_4 = SelfTrainingClassifier(base_lr, threshold=0.8, max_iter=100)
self_training_model_4.fit(X_mix_4, y_mix_4)

probs_4 = self_training_model_4.predict_proba(X_test)[:, 1]
auc_4 = roc_auc_score(y_test, probs_4)
print(f"AUC-ROC: {auc_4:.4f}")

print(f"1. Supervised (Baseline):          {auc_1:.4f}")
print(f"2. Semi-supervised (U != Test):    {auc_2:.4f}")
print(f"3. Semi-supervised (U == Test):    {auc_3:.4f}")
print(f"4. Semi-supervised (U incl. Test): {auc_4:.4f}")


1. Обучение только на размеченных данных (Supervised)
AUC-ROC: 0.8253
2. Semi-supervised (Unlabeled из Train, Test отделен)
AUC-ROC: 0.8210
3. Semi-supervised (Unlabeled == Test)
AUC-ROC: 0.8260
Сценарий 4: Semi-supervised (Unlabeled включает Test)
AUC-ROC: 0.8035
1. Supervised (Baseline):          0.8253
2. Semi-supervised (U != Test):    0.8210
3. Semi-supervised (U == Test):    0.8260
4. Semi-supervised (U incl. Test): 0.8035


Нас будут интересовать качество (AUC-ROC) в четырёх следующих постановках:
1. Модель обучается только на размеченных данных.
2. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть не пересекается с тестовой выборкой.
3. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть совпадает с тестовой выборкой.
4. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть включает в себя тестовую выборку.

**Задание 1. (5 баллов)**

Проведите эксперименты и сделайте выводы для любого из методов пакета `sklearn.semi_supervised` и для логистической регрессии

In [ ]:
#code here

### Self-train

Обучаем на размеченной части, предсказываем неразмеченную, потом обучаем на всех, и предсказываем неразмеченную, повторяем пока не сойдёмся в предскзааниях неразмеченной части